In [1]:
import wandb
import torch
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments, PushToHubCallback
from huggingface_hub import HfFolder

# Log in to Hugging Face
HfFolder.save_token(key)

# Initialize wandb
wandb.init(project='model_finetuning')

wandb: Currently logged in as: marcomolinari4. Use `wandb login --relogin` to force relogin


In [2]:
# Load the dataset
dataset = load_dataset("ArtifactAI/arxiv_python_research_code", split='train[:100000]')

Resolving data files:   0%|          | 0/26 [00:00<?, ?it/s]

In [3]:
# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("budecosystem/code-millenials-1b")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [4]:
# Tokenize the dataset
def tokenize_function(examples):
    return tokenizer(examples['code'], truncation=True, padding='max_length', max_length=128)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

In [5]:
model = AutoModelForCausalLM.from_pretrained("budecosystem/code-millenials-1b", trust_remote_code=True)

In [6]:
from transformers import DataCollatorForLanguageModeling

tokenizer.pad_token = tokenizer.eos_token
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

In [10]:
# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    num_train_epochs=1,
    per_device_train_batch_size=32,
    lr_scheduler_type="cosine",
    warmup_ratio=0.1,
    optim="adamw_torch",
    fp16=True,
    report_to="wandb",
    push_to_hub=True,
    hub_strategy="end",
    save_strategy="no",  # Prevents local saving of the model checkpoints.
    logging_dir="./logs",  # Optional: Specify where to log events locally.
    logging_steps=10,  # Log metrics every 10 steps. Adjust as needed to see logs more frequently.
    log_level='info'  # Set log level to 'info' to ensure detailed logs are visible.
)

PyTorch: setting up devices


In [11]:
# Initialize Trainer with PushToHubCallback for automatic model pushing
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets,
    eval_dataset=tokenized_datasets,
    data_collator=data_collator
)

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
Using auto half precision backend


In [12]:
# Start training
trainer.train()

The following columns in the training set don't have a corresponding argument in `PhiForCausalLM.forward` and have been ignored: file_length, file, extension_type, avg_line_length, code, repo, max_line_length. If file_length, file, extension_type, avg_line_length, code, repo, max_line_length are not expected by `PhiForCausalLM.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 100,000
  Num Epochs = 1
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 3,125
  Number of trainable parameters = 1,418,270,720
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


Epoch,Training Loss,Validation Loss
1,0.812100,nan


ValueError: Trainer: evaluation requires an eval_dataset.

In [13]:
model.push_to_hub("marco-molinari/results")

Configuration saved in results/config.json
Configuration saved in results/generation_config.json
The model is bigger than the maximum size per checkpoint (5GB) and is going to be split in 2 checkpoint shards. You can find where each parameters has been saved in the index located at results/model.safetensors.index.json.
Uploading the following files to marco-molinari/results: README.md,model.safetensors.index.json,config.json,model-00002-of-00002.safetensors,generation_config.json,model-00001-of-00002.safetensors


model-00001-of-00002.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/688M [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

CommitInfo(commit_url='https://huggingface.co/marco-molinari/results/commit/f3a84cd929b561ae34208c188a2b84681d49ad57', commit_message='Upload PhiForCausalLM', commit_description='', oid='f3a84cd929b561ae34208c188a2b84681d49ad57', pr_url=None, pr_revision=None, pr_num=None)